In [1]:
import numpy as np
import os
import datetime
import requests
import shutil
import zipfile
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from scripts.reconstruction_minimal import createAudio
import scipy.io.wavfile as wavfile
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Data prepare

In [28]:
def prepare_cnn(data, spectrogram, window_size):
    input = np.zeros((data.shape[0] - window_size, window_size, data.shape[1]))
    output = np.zeros((spectrogram.shape[0] - window_size, spectrogram.shape[1]))
    for i in range(data.shape[0] - window_size):
      output[i, :] = spectrogram[i + window_size, :]
      input[i, :, :] = data[i:i+window_size, :]
        
    return (input, output)

In [32]:
def prepare_cnn_dataset(X_train, Y_train, X_test, Y_test, window_size):
    for i in range(len(X_train)):
        (X_train[i], Y_train[i]) = prepare_cnn(X_train[i], Y_train[i], window_size)
        (X_test[i], Y_test[i]) = prepare_cnn(X_test[i], Y_test[i], window_size)

In [33]:
def generate_datasets(pt, feat_path=r'./features'):
    
    r"""
    Generate train and test datasets from the one sample of the raw data.
    param pt: name of the sample
    param feat_path: path of the raw data file
  """
    
    spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))
    data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))

    # Generate train, validation and test datasets
    X_train, X_test, Y_train, Y_test = train_test_split(data, spectrogram, test_size=0.1, shuffle=False)

    # Shuffle train data
    randomize = np.arange(len(X_train))
    np.random.shuffle(randomize)
    X_train = X_train[randomize]
    Y_train = Y_train[randomize]
    
    # Stnadardize input
    mu_input = np.mean(X_train, axis=0)
    std_input = np.std(X_train, axis=0)
    X_train = (X_train-mu_input)/std_input
    X_test = (X_test-mu_input)/std_input

    # Reduce Dimensions
    pca = PCA()
    pca.fit(X_train)
    X_train = np.dot(X_train, pca.components_[:50,:].T)
    X_test = np.dot(X_test, pca.components_[:50,:].T)

    # Standardize output
    mu_output = np.mean(Y_train, axis=0)
    std_output = np.std(Y_train, axis=0)
    Y_train = (Y_train-mu_output)/std_output
    Y_test = (Y_test-mu_output)/std_output


    return (X_train, Y_train, X_test, Y_test, mu_input, std_input, mu_output, std_output)

In [37]:
def generate_train_test_datasets():
    norm_params = {}
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    pts = ['sub-%02d'%i for i in range(1, 11)]
    for i, pt in enumerate(pts):
        (x_train, y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pt)
        norm_params[str(i)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
        X_train.append(x_train)
        Y_train.append(y_train)
        X_test.append(x_test)
        Y_test.append(y_test)
    return (X_train, Y_train, X_test, Y_test, norm_params)

# Modelling

In [7]:
# Synthesize audio files
def synthesize_audio(pred, spectogram, result_path, result_name):
    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000
    if not os.path.isdir(result_path):
        os.makedirs(result_path)
    
    #Synthesize waveform from spectrogram using Griffin-Lim
    reconstructedWav = createAudio(pred,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
    wavfile.write(os.path.join(result_path,f'{result_name}_predicted.wav'),int(audiosr),reconstructedWav)

    #For comparison synthesize the original spectrogram with Griffin-Lim
    origWav = createAudio(spectogram,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
    wavfile.write(os.path.join(result_path,f'{result_name}_orig_synthesized.wav'),int(audiosr),origWav)

In [ ]:
# Viz spectrograms
def visualize_results(preds, spectograms, result_path, result_name):

    pts_num = len(spectograms)
    cm='viridis'
    fig, ax = plt.subplots(2, sharex=True)

    if not os.path.isdir(result_path):
        os.makedirs(result_path)

    for i in range(pts_num):
        #Plot spectrograms
        ax[0].imshow(np.flipud(spectograms[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[0].set_ylabel('Log Mel-Spec Bin - original')
        ax[1].imshow(np.flipud(preds[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[1].set_ylabel('Log Mel-Spec Bin - prediction')
        plt.savefig(os.path.join(result_path,f'{result_name}_spectograms.jpg'))

In [8]:
def create_fndnn(input_dim, output_dim, lr=0.001, betas=(0.9, 0.99)):
    model = Sequential()
    model.add(Dense(units=60, input_dim=input_dim, activation='relu'))
    model.add(Dense(units=30, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=lr, beta_1=betas[0], beta_2=betas[1])
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [46]:
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=[40, 40, 40], lr=0.001, betas=(0.9, 0.99)):
    #probably more filter layers should be added

    model = Sequential()
    model.add(Conv1D(filters=nb_filter[0], kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)))
    model.add(Conv1D(filters=nb_filter[1], kernel_size=filter_length, activation='relu'))
    model.add(Conv1D(filters=nb_filter[2], kernel_size=filter_length, activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=30, activation='relu'))
    model.add(Dense(nb_outputs, activation='linear'))

    optimizer=Adam(learning_rate=lr, beta_1=betas[0], beta_2=betas[1])
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [35]:
def train_test_models(model, X_train, Y_train, X_test, Y_test, result_path,
    epochs = 1000, batch_size = 512, patience = 50, lr = 0.001, betas = (0.99, 0.999)):
    for i in range(len(X_test)):
        log_folder = f"logs/sample{i+1}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        early_stopping=EarlyStopping(patience=patience, verbose=1)
        checkpointer=ModelCheckpoint(filepath='fcdnn.hdf5', save_best_only=True, verbose=1)
        tensorboard = TensorBoard(log_dir=log_folder, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch', profile_batch=2, embeddings_freq=1)
        
        print(f"Training for sample: {i+1}")
        model.fit(X_train[i], Y_train[i], validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping, tensorboard])
        y_pred = model.predict(X_test[i])
        mae = mean_absolute_error(Y_test[i], y_pred)
        print(f'Mean absolute error of test set:{mae}\n\n')
        
        pred = y_pred * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output']
        spectogram = Y_test[i] * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output']
        synthesize_audio(pred, spectogram, result_path=result_path, result_name=f"sub-{i+1}")
        visualize_results(preds=pred, spectograms=spectogram, log_folder=log_folder)

        text = 'Optimization hyperparameters:' + str(dict(alg='adam', batch_size=batch_size, patience=patience, lr=lr, betas=betas)) + \
        '\n'
        from contextlib import redirect_stdout
        with open(log_folder + '/nfo.txt', 'w') as f:
                f.write(text)
        with open(log_folder + '/nfo.txt', 'a') as f:
            with redirect_stdout(f):
                model_cnn.summary()
    

In [ ]:
epochs = 1000
batch_size = 512
patience = 50
lr = 0.001
betas = (0.99, 0.999)
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()

In [11]:
model = create_fndnn(input_dim=X_train[0].shape[1], output_dim=Y_train[0].shape[1])
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="dnn_results", epochs = epochs, batch_size = batch_size, patience = patience, lr = lr, betas = betas)

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.99174, saving model to fcdnn.hdf5
48/48 - 2s - loss: 1.5236 - mae: 0.9162 - val_loss: 0.9917 - val_mae: 0.7574 - 2s/epoch - 34ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 0.99174 to 0.96994, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0130 - mae: 0.7744 - val_loss: 0.9699 - val_mae: 0.7531 - 346ms/epoch - 7ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 0.96994 to 0.96656, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0044 - mae: 0.7721 - val_loss: 0.9666 - val_mae: 0.7518 - 377ms/epoch - 8ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 0.96656 to 0.96193, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0003 - mae: 0.7707 - val_loss: 0.9619 - val_mae: 0.7503 - 377ms/epoch - 8ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 0.96193 to 0.94340, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9890 - mae: 0.7658 - val_loss: 0.9434 - val_mae: 0.7467 - 298ms/epoch - 6ms/step
Epoch 6/

Epoch 44/1000

Epoch 44: val_loss improved from 0.41628 to 0.41579, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3030 - mae: 0.3684 - val_loss: 0.4158 - val_mae: 0.4335 - 393ms/epoch - 8ms/step
Epoch 45/1000

Epoch 45: val_loss improved from 0.41579 to 0.41090, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.3018 - mae: 0.3678 - val_loss: 0.4109 - val_mae: 0.4298 - 408ms/epoch - 9ms/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 0.41090
48/48 - 0s - loss: 0.2985 - mae: 0.3652 - val_loss: 0.4146 - val_mae: 0.4288 - 343ms/epoch - 7ms/step
Epoch 47/1000

Epoch 47: val_loss did not improve from 0.41090
48/48 - 0s - loss: 0.2962 - mae: 0.3636 - val_loss: 0.4136 - val_mae: 0.4306 - 282ms/epoch - 6ms/step
Epoch 48/1000

Epoch 48: val_loss did not improve from 0.41090
48/48 - 0s - loss: 0.2924 - mae: 0.3624 - val_loss: 0.4207 - val_mae: 0.4345 - 346ms/epoch - 7ms/step
Epoch 49/1000

Epoch 49: val_loss improved from 0.41090 to 0.40996, saving model to fcdnn.hdf5
48/48 - 0s - los

Epoch 90/1000

Epoch 90: val_loss did not improve from 0.35414
48/48 - 0s - loss: 0.2279 - mae: 0.3181 - val_loss: 0.3569 - val_mae: 0.3920 - 263ms/epoch - 5ms/step
Epoch 91/1000

Epoch 91: val_loss improved from 0.35414 to 0.34980, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.2276 - mae: 0.3184 - val_loss: 0.3498 - val_mae: 0.3870 - 348ms/epoch - 7ms/step
Epoch 92/1000

Epoch 92: val_loss did not improve from 0.34980
48/48 - 0s - loss: 0.2258 - mae: 0.3166 - val_loss: 0.3541 - val_mae: 0.3901 - 300ms/epoch - 6ms/step
Epoch 93/1000

Epoch 93: val_loss did not improve from 0.34980
48/48 - 0s - loss: 0.2251 - mae: 0.3168 - val_loss: 0.3499 - val_mae: 0.3859 - 300ms/epoch - 6ms/step
Epoch 94/1000

Epoch 94: val_loss did not improve from 0.34980
48/48 - 0s - loss: 0.2246 - mae: 0.3163 - val_loss: 0.3547 - val_mae: 0.3958 - 268ms/epoch - 6ms/step
Epoch 95/1000

Epoch 95: val_loss did not improve from 0.34980
48/48 - 0s - loss: 0.2245 - mae: 0.3161 - val_loss: 0.3580 - val_mae: 0.3908 - 3

Epoch 137/1000

Epoch 137: val_loss improved from 0.32977 to 0.32976, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.1981 - mae: 0.2981 - val_loss: 0.3298 - val_mae: 0.3735 - 329ms/epoch - 7ms/step
Epoch 138/1000

Epoch 138: val_loss did not improve from 0.32976
48/48 - 0s - loss: 0.1968 - mae: 0.2973 - val_loss: 0.3303 - val_mae: 0.3761 - 355ms/epoch - 7ms/step
Epoch 139/1000

Epoch 139: val_loss did not improve from 0.32976
48/48 - 0s - loss: 0.1967 - mae: 0.2970 - val_loss: 0.3335 - val_mae: 0.3767 - 288ms/epoch - 6ms/step
Epoch 140/1000

Epoch 140: val_loss did not improve from 0.32976
48/48 - 0s - loss: 0.1967 - mae: 0.2976 - val_loss: 0.3343 - val_mae: 0.3790 - 299ms/epoch - 6ms/step
Epoch 141/1000

Epoch 141: val_loss did not improve from 0.32976
48/48 - 0s - loss: 0.1962 - mae: 0.2968 - val_loss: 0.3334 - val_mae: 0.3764 - 269ms/epoch - 6ms/step
Epoch 142/1000

Epoch 142: val_loss did not improve from 0.32976
48/48 - 0s - loss: 0.1986 - mae: 0.2987 - val_loss: 0.3322 - val_mae

48/48 - 0s - loss: 0.1848 - mae: 0.2888 - val_loss: 0.3220 - val_mae: 0.3656 - 374ms/epoch - 8ms/step
Epoch 185/1000

Epoch 185: val_loss improved from 0.31816 to 0.31802, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.1897 - mae: 0.2919 - val_loss: 0.3180 - val_mae: 0.3695 - 364ms/epoch - 8ms/step
Epoch 186/1000

Epoch 186: val_loss did not improve from 0.31802
48/48 - 0s - loss: 0.1831 - mae: 0.2873 - val_loss: 0.3216 - val_mae: 0.3703 - 330ms/epoch - 7ms/step
Epoch 187/1000

Epoch 187: val_loss improved from 0.31802 to 0.31746, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.1819 - mae: 0.2861 - val_loss: 0.3175 - val_mae: 0.3698 - 325ms/epoch - 7ms/step
Epoch 188/1000

Epoch 188: val_loss improved from 0.31746 to 0.31436, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.1843 - mae: 0.2879 - val_loss: 0.3144 - val_mae: 0.3677 - 283ms/epoch - 6ms/step
Epoch 189/1000

Epoch 189: val_loss did not improve from 0.31436
48/48 - 0s - loss: 0.1822 - mae: 0.2865 - val_loss: 0.3165 - val_mae:

...

Epoch 600/1000

Epoch 600: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0914 - mae: 0.2054 - val_loss: 0.1314 - val_mae: 0.2383 - 284ms/epoch - 6ms/step
Epoch 601/1000

Epoch 601: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0922 - mae: 0.2060 - val_loss: 0.1308 - val_mae: 0.2371 - 393ms/epoch - 9ms/step
Epoch 602/1000

Epoch 602: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0931 - mae: 0.2067 - val_loss: 0.1358 - val_mae: 0.2410 - 324ms/epoch - 7ms/step
Epoch 603/1000

Epoch 603: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0965 - mae: 0.2090 - val_loss: 0.1321 - val_mae: 0.2382 - 290ms/epoch - 6ms/step
Epoch 604/1000

Epoch 604: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0930 - mae: 0.2068 - val_loss: 0.1317 - val_mae: 0.2382 - 282ms/epoch - 6ms/step
Epoch 605/1000

Epoch 605: val_loss did not improve from 0.13035
46/46 - 0s - loss: 0.0916 - mae: 0.2056 - val_loss: 0.1325 - val_mae: 0.2385 - 299ms/epoch - 7ms/ste

Epoch 649/1000

Epoch 649: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0906 - mae: 0.2047 - val_loss: 0.1300 - val_mae: 0.2371 - 300ms/epoch - 7ms/step
Epoch 650/1000

Epoch 650: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0915 - mae: 0.2055 - val_loss: 0.1304 - val_mae: 0.2369 - 283ms/epoch - 6ms/step
Epoch 651/1000

Epoch 651: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0913 - mae: 0.2052 - val_loss: 0.1315 - val_mae: 0.2377 - 299ms/epoch - 6ms/step
Epoch 652/1000

Epoch 652: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0917 - mae: 0.2058 - val_loss: 0.1306 - val_mae: 0.2367 - 299ms/epoch - 6ms/step
Epoch 653/1000

Epoch 653: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0911 - mae: 0.2050 - val_loss: 0.1295 - val_mae: 0.2358 - 298ms/epoch - 6ms/step
Epoch 654/1000

Epoch 654: val_loss did not improve from 0.12872
46/46 - 0s - loss: 0.0919 - mae: 0.2061 - val_loss: 0.1301 - val_mae: 0.2372 - 283ms/epoch - 6ms/ste

Epoch 698/1000

Epoch 698: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0896 - mae: 0.2037 - val_loss: 0.1292 - val_mae: 0.2365 - 425ms/epoch - 9ms/step
Epoch 699/1000

Epoch 699: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0894 - mae: 0.2036 - val_loss: 0.1302 - val_mae: 0.2365 - 315ms/epoch - 7ms/step
Epoch 700/1000

Epoch 700: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0904 - mae: 0.2043 - val_loss: 0.1312 - val_mae: 0.2365 - 392ms/epoch - 9ms/step
Epoch 701/1000

Epoch 701: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0920 - mae: 0.2060 - val_loss: 0.1293 - val_mae: 0.2356 - 252ms/epoch - 5ms/step
Epoch 702/1000

Epoch 702: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0904 - mae: 0.2044 - val_loss: 0.1295 - val_mae: 0.2356 - 284ms/epoch - 6ms/step
Epoch 703/1000

Epoch 703: val_loss did not improve from 0.12841
46/46 - 0s - loss: 0.0896 - mae: 0.2035 - val_loss: 0.1300 - val_mae: 0.2359 - 410ms/epoch - 9ms/ste

Epoch 746/1000

Epoch 746: val_loss did not improve from 0.12690
46/46 - 0s - loss: 0.0894 - mae: 0.2032 - val_loss: 0.1287 - val_mae: 0.2350 - 297ms/epoch - 6ms/step
Epoch 747/1000

Epoch 747: val_loss did not improve from 0.12690
46/46 - 0s - loss: 0.0894 - mae: 0.2031 - val_loss: 0.1271 - val_mae: 0.2345 - 349ms/epoch - 8ms/step
Epoch 748/1000

Epoch 748: val_loss did not improve from 0.12690
46/46 - 0s - loss: 0.0978 - mae: 0.2091 - val_loss: 0.1272 - val_mae: 0.2349 - 315ms/epoch - 7ms/step
Epoch 749/1000

Epoch 749: val_loss did not improve from 0.12690
46/46 - 0s - loss: 0.0921 - mae: 0.2053 - val_loss: 0.1305 - val_mae: 0.2380 - 395ms/epoch - 9ms/step
Epoch 750/1000

Epoch 750: val_loss improved from 0.12690 to 0.12610, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.0931 - mae: 0.2056 - val_loss: 0.1261 - val_mae: 0.2346 - 332ms/epoch - 7ms/step
Epoch 751/1000

Epoch 751: val_loss improved from 0.12610 to 0.12495, saving model to fcdnn.hdf5
46/46 - 0s - loss: 0.0902 - mae: 0.2

Epoch 795/1000

Epoch 795: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0883 - mae: 0.2023 - val_loss: 0.1256 - val_mae: 0.2331 - 351ms/epoch - 8ms/step
Epoch 796/1000

Epoch 796: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0881 - mae: 0.2020 - val_loss: 0.1295 - val_mae: 0.2352 - 276ms/epoch - 6ms/step
Epoch 797/1000

Epoch 797: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0888 - mae: 0.2027 - val_loss: 0.1269 - val_mae: 0.2344 - 363ms/epoch - 8ms/step
Epoch 798/1000

Epoch 798: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0885 - mae: 0.2024 - val_loss: 0.1272 - val_mae: 0.2346 - 347ms/epoch - 8ms/step
Epoch 799/1000

Epoch 799: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0882 - mae: 0.2024 - val_loss: 0.1258 - val_mae: 0.2334 - 257ms/epoch - 6ms/step
Epoch 800/1000

Epoch 800: val_loss did not improve from 0.12495
46/46 - 0s - loss: 0.0890 - mae: 0.2026 - val_loss: 0.1259 - val_mae: 0.2333 - 255ms/epoch - 6ms/ste

In [ ]:
epochs = 1000
batch_size = 512
patience = 50
lr = 0.001
betas = (0.99, 0.999)
window_size = 8
nb_filters=[40, 40, 40]
filter_length=2
(X_train, Y_train, X_test, Y_test, norm_params) = generate_train_test_datasets()

In [48]:
model = make_1d_convnet(window_size=window_size, filter_length=2, nb_filter=nb_filters, nb_input_series=50, nb_outputs=23)
prepare_cnn_dataset(X_train, Y_train, X_test, Y_test, window_size=window_size)
train_test_models(model, X_train, Y_train, X_test, Y_test, result_path="cnn_results", 
epochs = epochs, batch_size = batch_size, patience = patience, lr = lr, betas = betas)

Training for sample: 1
Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.02311, saving model to fcdnn.hdf5
48/48 - 2s - loss: 1.1540 - mae: 0.8158 - val_loss: 1.0231 - val_mae: 0.7713 - 2s/epoch - 44ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 1.02311 to 1.01323, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.0081 - mae: 0.7708 - val_loss: 1.0132 - val_mae: 0.7698 - 582ms/epoch - 12ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.01323 to 1.01015, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.0023 - mae: 0.7695 - val_loss: 1.0102 - val_mae: 0.7704 - 551ms/epoch - 11ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.01015 to 1.00905, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.0002 - mae: 0.7701 - val_loss: 1.0090 - val_mae: 0.7705 - 787ms/epoch - 16ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 1.00905 to 1.00870, saving model to fcdnn.hdf5
48/48 - 0s - loss: 0.9993 - mae: 0.7695 - val_loss: 1.0087 - val_mae: 0.7706 - 488ms/epoch - 10ms/step
Epoc

Epoch 50/1000

Epoch 50: val_loss did not improve from 1.00870
48/48 - 0s - loss: 0.5175 - mae: 0.5180 - val_loss: 1.4700 - val_mae: 0.8814 - 431ms/epoch - 9ms/step
Epoch 51/1000

Epoch 51: val_loss did not improve from 1.00870
48/48 - 1s - loss: 0.5113 - mae: 0.5139 - val_loss: 1.4236 - val_mae: 0.8622 - 661ms/epoch - 14ms/step
Epoch 52/1000

Epoch 52: val_loss did not improve from 1.00870
48/48 - 1s - loss: 0.5045 - mae: 0.5099 - val_loss: 1.4600 - val_mae: 0.8765 - 644ms/epoch - 13ms/step
Epoch 53/1000

Epoch 53: val_loss did not improve from 1.00870
48/48 - 1s - loss: 0.5027 - mae: 0.5089 - val_loss: 1.4425 - val_mae: 0.8659 - 506ms/epoch - 11ms/step
Epoch 54/1000

Epoch 54: val_loss did not improve from 1.00870
48/48 - 0s - loss: 0.4966 - mae: 0.5050 - val_loss: 1.4648 - val_mae: 0.8738 - 457ms/epoch - 10ms/step
Epoch 55/1000

Epoch 55: val_loss did not improve from 1.00870
48/48 - 1s - loss: 0.4938 - mae: 0.5035 - val_loss: 1.4605 - val_mae: 0.8663 - 677ms/epoch - 14ms/step
Epoch


Epoch 43: val_loss did not improve from 1.01134
48/48 - 0s - loss: 0.6162 - mae: 0.5732 - val_loss: 1.3412 - val_mae: 0.8625 - 424ms/epoch - 9ms/step
Epoch 44/1000

Epoch 44: val_loss did not improve from 1.01134
48/48 - 1s - loss: 0.6065 - mae: 0.5681 - val_loss: 1.3531 - val_mae: 0.8626 - 539ms/epoch - 11ms/step
Epoch 45/1000

Epoch 45: val_loss did not improve from 1.01134
48/48 - 1s - loss: 0.6044 - mae: 0.5660 - val_loss: 1.3632 - val_mae: 0.8717 - 591ms/epoch - 12ms/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 1.01134
48/48 - 1s - loss: 0.5958 - mae: 0.5609 - val_loss: 1.3853 - val_mae: 0.8855 - 754ms/epoch - 16ms/step
Epoch 47/1000

Epoch 47: val_loss did not improve from 1.01134
48/48 - 1s - loss: 0.5930 - mae: 0.5621 - val_loss: 1.3848 - val_mae: 0.8681 - 507ms/epoch - 11ms/step
Epoch 48/1000

Epoch 48: val_loss did not improve from 1.01134
48/48 - 1s - loss: 0.5881 - mae: 0.5583 - val_loss: 1.4111 - val_mae: 0.8763 - 739ms/epoch - 15ms/step
Epoch 49/1000

Epoc

48/48 - 1s - loss: 0.7601 - mae: 0.6719 - val_loss: 1.2240 - val_mae: 0.8766 - 661ms/epoch - 14ms/step
Epoch 39/1000

Epoch 39: val_loss did not improve from 1.02370
48/48 - 1s - loss: 0.7495 - mae: 0.6656 - val_loss: 1.2210 - val_mae: 0.8612 - 669ms/epoch - 14ms/step
Epoch 40/1000

Epoch 40: val_loss did not improve from 1.02370
48/48 - 1s - loss: 0.7444 - mae: 0.6626 - val_loss: 1.2385 - val_mae: 0.8750 - 660ms/epoch - 14ms/step
Epoch 41/1000

Epoch 41: val_loss did not improve from 1.02370
48/48 - 1s - loss: 0.7375 - mae: 0.6604 - val_loss: 1.2570 - val_mae: 0.8602 - 608ms/epoch - 13ms/step
Epoch 42/1000

Epoch 42: val_loss did not improve from 1.02370
48/48 - 1s - loss: 0.7353 - mae: 0.6552 - val_loss: 1.2513 - val_mae: 0.8851 - 530ms/epoch - 11ms/step
Epoch 43/1000

Epoch 43: val_loss did not improve from 1.02370
48/48 - 0s - loss: 0.7297 - mae: 0.6550 - val_loss: 1.2611 - val_mae: 0.8650 - 454ms/epoch - 9ms/step
Epoch 44/1000

Epoch 44: val_loss did not improve from 1.02370
48/48

Epoch 33/1000

Epoch 33: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.9129 - mae: 0.7451 - val_loss: 1.0551 - val_mae: 0.8098 - 649ms/epoch - 14ms/step
Epoch 34/1000

Epoch 34: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.9074 - mae: 0.7422 - val_loss: 1.0626 - val_mae: 0.7990 - 595ms/epoch - 12ms/step
Epoch 35/1000

Epoch 35: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.9029 - mae: 0.7412 - val_loss: 1.0781 - val_mae: 0.8093 - 629ms/epoch - 13ms/step
Epoch 36/1000

Epoch 36: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.8966 - mae: 0.7360 - val_loss: 1.0763 - val_mae: 0.7776 - 583ms/epoch - 12ms/step
Epoch 37/1000

Epoch 37: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.8916 - mae: 0.7327 - val_loss: 1.0764 - val_mae: 0.7967 - 664ms/epoch - 14ms/step
Epoch 38/1000

Epoch 38: val_loss did not improve from 1.00408
48/48 - 1s - loss: 0.8844 - mae: 0.7294 - val_loss: 1.0917 - val_mae: 0.8208 - 632ms/epoch - 13ms/step
Epoc

...

Epoch 15/1000

Epoch 15: val_loss improved from 0.97999 to 0.97972, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.0005 - mae: 0.7476 - val_loss: 0.9797 - val_mae: 0.7262 - 507ms/epoch - 11ms/step
Epoch 16/1000

Epoch 16: val_loss did not improve from 0.97972
48/48 - 0s - loss: 1.0005 - mae: 0.7443 - val_loss: 0.9833 - val_mae: 0.7495 - 474ms/epoch - 10ms/step
Epoch 17/1000

Epoch 17: val_loss did not improve from 0.97972
48/48 - 0s - loss: 1.0001 - mae: 0.7450 - val_loss: 0.9805 - val_mae: 0.7245 - 432ms/epoch - 9ms/step
Epoch 18/1000

Epoch 18: val_loss did not improve from 0.97972
48/48 - 0s - loss: 0.9998 - mae: 0.7432 - val_loss: 0.9804 - val_mae: 0.7314 - 431ms/epoch - 9ms/step
Epoch 19/1000

Epoch 19: val_loss did not improve from 0.97972
48/48 - 1s - loss: 0.9996 - mae: 0.7432 - val_loss: 0.9811 - val_mae: 0.7420 - 692ms/epoch - 14ms/step
Epoch 20/1000

Epoch 20: val_loss did not improve from 0.97972
48/48 - 1s - loss: 0.9991 - mae: 0.7447 - val_loss: 0.9846 - val_mae: 0.7559 

48/48 - 0s - loss: 0.9800 - mae: 0.7349 - val_loss: 0.9929 - val_mae: 0.7430 - 498ms/epoch - 10ms/step
Epoch 65/1000

Epoch 65: val_loss did not improve from 0.97972
48/48 - 0s - loss: 0.9803 - mae: 0.7328 - val_loss: 1.0070 - val_mae: 0.7545 - 473ms/epoch - 10ms/step
Epoch 65: early stopping
94/94 [==============================] - 0s 1ms/step
Mean absolute error of test set:0.7800784723009302


Training for sample: 8
Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.97536, saving model to fcdnn.hdf5
48/48 - 1s - loss: 1.0081 - mae: 0.7640 - val_loss: 0.9754 - val_mae: 0.7572 - 911ms/epoch - 19ms/step
Epoch 2/1000

Epoch 2: val_loss did not improve from 0.97536
48/48 - 0s - loss: 1.0042 - mae: 0.7647 - val_loss: 0.9771 - val_mae: 0.7716 - 441ms/epoch - 9ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 0.97536 to 0.97501, saving model to fcdnn.hdf5
48/48 - 0s - loss: 1.0029 - mae: 0.7666 - val_loss: 0.9750 - val_mae: 0.7609 - 442ms/epoch - 9ms/step
Epoch 4/1000

Epoch 4: val_

Epoch 48/1000

Epoch 48: val_loss did not improve from 0.97464
48/48 - 1s - loss: 0.8058 - mae: 0.6527 - val_loss: 1.2227 - val_mae: 0.8481 - 663ms/epoch - 14ms/step
Epoch 49/1000

Epoch 49: val_loss did not improve from 0.97464
48/48 - 1s - loss: 0.7960 - mae: 0.6450 - val_loss: 1.2228 - val_mae: 0.8292 - 670ms/epoch - 14ms/step
Epoch 50/1000

Epoch 50: val_loss did not improve from 0.97464
48/48 - 1s - loss: 0.7908 - mae: 0.6422 - val_loss: 1.2192 - val_mae: 0.8247 - 520ms/epoch - 11ms/step
Epoch 51/1000

Epoch 51: val_loss did not improve from 0.97464
48/48 - 0s - loss: 0.7816 - mae: 0.6361 - val_loss: 1.2276 - val_mae: 0.8420 - 473ms/epoch - 10ms/step
Epoch 52/1000

Epoch 52: val_loss did not improve from 0.97464
48/48 - 1s - loss: 0.7776 - mae: 0.6346 - val_loss: 1.2431 - val_mae: 0.8294 - 578ms/epoch - 12ms/step
Epoch 53/1000

Epoch 53: val_loss did not improve from 0.97464
48/48 - 1s - loss: 0.7711 - mae: 0.6312 - val_loss: 1.2408 - val_mae: 0.8164 - 531ms/epoch - 11ms/step
Epoc


Epoch 41: val_loss did not improve from 1.00031
48/48 - 1s - loss: 0.9078 - mae: 0.6976 - val_loss: 1.0876 - val_mae: 0.7572 - 552ms/epoch - 12ms/step
Epoch 42/1000

Epoch 42: val_loss did not improve from 1.00031
48/48 - 0s - loss: 0.9027 - mae: 0.6931 - val_loss: 1.0830 - val_mae: 0.7590 - 458ms/epoch - 10ms/step
Epoch 43/1000

Epoch 43: val_loss did not improve from 1.00031
48/48 - 0s - loss: 0.9006 - mae: 0.6936 - val_loss: 1.0874 - val_mae: 0.7732 - 495ms/epoch - 10ms/step
Epoch 44/1000

Epoch 44: val_loss did not improve from 1.00031
48/48 - 1s - loss: 0.8950 - mae: 0.6903 - val_loss: 1.0906 - val_mae: 0.7471 - 559ms/epoch - 12ms/step
Epoch 45/1000

Epoch 45: val_loss did not improve from 1.00031
48/48 - 1s - loss: 0.8854 - mae: 0.6860 - val_loss: 1.0950 - val_mae: 0.7496 - 590ms/epoch - 12ms/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 1.00031
48/48 - 1s - loss: 0.8741 - mae: 0.6794 - val_loss: 1.0972 - val_mae: 0.7654 - 620ms/epoch - 13ms/step
Epoch 47/1000

Epo

Epoch 36/1000

Epoch 36: val_loss did not improve from 1.00361
46/46 - 0s - loss: 0.8981 - mae: 0.6175 - val_loss: 1.0610 - val_mae: 0.6778 - 462ms/epoch - 10ms/step
Epoch 37/1000

Epoch 37: val_loss did not improve from 1.00361
46/46 - 0s - loss: 0.8859 - mae: 0.6097 - val_loss: 1.0569 - val_mae: 0.6712 - 488ms/epoch - 11ms/step
Epoch 38/1000

Epoch 38: val_loss did not improve from 1.00361
46/46 - 0s - loss: 0.8794 - mae: 0.6090 - val_loss: 1.0788 - val_mae: 0.6861 - 431ms/epoch - 9ms/step
Epoch 39/1000

Epoch 39: val_loss did not improve from 1.00361
46/46 - 0s - loss: 0.8767 - mae: 0.6090 - val_loss: 1.0753 - val_mae: 0.6775 - 404ms/epoch - 9ms/step
Epoch 40/1000

Epoch 40: val_loss did not improve from 1.00361
46/46 - 1s - loss: 0.8680 - mae: 0.6034 - val_loss: 1.0785 - val_mae: 0.6750 - 550ms/epoch - 12ms/step
Epoch 41/1000

Epoch 41: val_loss did not improve from 1.00361
46/46 - 1s - loss: 0.8658 - mae: 0.6006 - val_loss: 1.0866 - val_mae: 0.6594 - 504ms/epoch - 11ms/step
Epoch 